This script creates the functional tool to create the Dash web app to project all provider network graph on to a webpage. Not all of the below code is functional. 


In [3]:
import dash
from dash import html
from dash import dcc
import numpy as np
import pandas as pd
import dash_cytoscape as cyto
import networkx as nx
import matplotlib as mpl # see colorFader func.
#from sklearn.preprocessing import StandardScaler # rescale numbers/ normalise data
from dash.dependencies import Input, Output # for callbacks
from sklearn.preprocessing import StandardScaler

In [5]:
#create the graph including some basic total interaction statistic for each node. 
def create_nx_graph(OrgCode):
    nodeData = pd.read_csv('node_list.csv', low_memory=False)
    edgeData = pd.read_csv('edge_list/edge_list_' +OrgCode+'.csv', low_memory=False)
    nodeData = nodeData.astype(str)
    edgeData = edgeData.astype(str)
    
    total_interactions_node = pd.concat([edgeData[['Source', 'Weight']].groupby('Source').sum().reset_index(drop=False),
                                     edgeData[['Target', 'Weight']].groupby('Target').sum().reset_index(drop=False).rename(columns={'Target':'Source'})]
                                     ).groupby('Source').sum().rename(columns={'Weight':'TotalInteractions'})

    nodeData = nodeData.merge(total_interactions_node, how="left", left_on="ID", right_on="Source")
 
    
    ## Initiate the graph object
    G = nx.Graph()
    
    ## Tranform the data into the correct format for use with NetworkX
    # Node tuples (ID, dict of attributes)
    idList = nodeData['ID'].tolist()
    labels =  pd.DataFrame(nodeData['Label'])
    labelDicts = labels.to_dict(orient='records')
    nodeTuples = [tuple(r) for r in zip(idList,labelDicts)]
    
    # Edge tuples (Source, Target, dict of attributes)
    sourceList = edgeData['Source'].tolist()
    targetList = edgeData['Target'].tolist()
    weights = pd.DataFrame(edgeData['Weight'])
    weightDicts = weights.to_dict(orient='records')
    edgeTuples = [tuple(r) for r in zip(sourceList,targetList,weightDicts)]
    
    # Add the nodes and edges to the graph
    # 'add_nodes_from' adds nodes from any iterable container
    G.add_nodes_from(nodeTuples)
    G.add_edges_from(edgeTuples)

    totalInteractionsDicts = {}
    totalInteractionsDicts.update(zip(nodeData['ID'], nodeData['TotalInteractions']))
    nx.set_node_attributes(G, totalInteractionsDicts, "TotalInteractions")
    
    G.remove_nodes_from(list(nx.isolates(G)))
    #set something no idea what
    bb = nx.betweenness_centrality(G)
    nx.set_node_attributes(G, bb, "Size")
    
    c = nx.community.greedy_modularity_communities(G)

    community_dict = pd.DataFrame(c).reset_index(drop=False).melt(id_vars="index")[['value','index']]
    community_dict = community_dict[community_dict['value'].notnull()]
    communityDicts = {}
    communityDicts.update(zip(community_dict['value'], community_dict['index']))

    nx.set_node_attributes(G, communityDicts, "Community")


    
    color_df = pd.DataFrame(
        [{'index': 0, 'color': "#fbf59a"},
        {'index': 1, 'color': "#674ea7"},
        {'index': 2, 'color': "#72a45d"},
        {'index': 3, 'color': "#f2600b"},
        {'index': 4, 'color': "#2986cc"}]
    )

    community_dict = community_dict.merge(color_df, how="left", on="index")

    communityColorDicts = {}
    communityColorDicts.update(zip(community_dict['value'], community_dict['color']))

    nx.set_node_attributes(G, communityColorDicts, "CommunityColor")
    # nx.set_node_attributes(G, colorDicts, "Color")

    return G

In [6]:
#This set of functions relate to the colour changes of the nodes not yet functional
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    """
    Creates a linear gradient colour fade between two colors 
        
    Parameters
    ----------
    c1 : str
        Colour value
    c2 : str
        Colour value
        
    mix : int (Default = 0)
        Value of 1 or 0

    Returns
    -------
    mpl.colors.to_hex((1-mix)*c1 + mix*c2) : str
        Hex colour value

    """
    
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

def node_color_picker(par_df,col_name):
    """
    Function to work out the HEX value of the colour required for the node
    depending on where it sits on the gradient range from green to blue

    Parameters
    ----------
    par_df : pd.DataFrame
        Dataframe containing connectivity values.
    col_name : str
        Name of column containing values.

    Returns
    -------
    par_col_df : pd.DataFrame
        Dataframe containing gradient colour value

    """
    par_max = par_df.max()
    par_max = par_max.iloc[0]
    c1='#1f77b4' #blue
    c2='green' #green
    n=par_max
    
    par_col_list = list()
    for i in range(len(par_df)):
        par_val = par_df.iloc[i,0]
        color=colorFader(c1,c2,par_val/n)
        par_col_list.append(color)
    
    par_col_df = pd.DataFrame(par_col_list, columns=[col_name])
    
    return par_col_df

In [7]:
#Creat the set of analysis this has slight fucntionality. 
def create_analysis(G):
    """
    Function to assemble different NetworkX analysis so they they can 
    be used when creating the Cytoscape graph elements.
    #### CHECK THIS!

    Parameters
    ----------
    G : nx.Graph
        NetworkX Graph object
    nodes : pd.DataFrame
        Node list
        ###############################
        THIS DOESNT APPEAR TO BE USED?
        ###############################

    Returns
    -------
    full_an_df : TYPE
        Single dataframe containing all metrics.

    """
    # Node metrics
    # Diff measures of connectivity
    e_cent = nx.eigenvector_centrality(G)
    page_rank = nx.pagerank(G)
    degree = nx.degree(G)
    between = nx.betweenness_centrality(G)
    
    # Extract the analysis output and convert to a suitable scale and format
    e_cent_size = pd.DataFrame.from_dict(e_cent, orient='index',
                                         columns=['cent_value'])
    e_cent_size.reset_index(drop=True, inplace=True)
    #e_cent_size = e_cent_size*100
    page_rank_size = pd.DataFrame.from_dict(page_rank, orient='index',
                                            columns=['rank_value'])
    page_rank_size.reset_index(drop=True, inplace=True)
    #page_rank_size = page_rank_size*1000
    degree_list = list(degree)
    degree_dict = dict(degree_list)
    degree_size = pd.DataFrame.from_dict(degree_dict, orient='index',
                                         columns=['deg_value'])
    degree_size.reset_index(drop=True, inplace=True)
    between_size = pd.DataFrame.from_dict(between, orient='index',
                                          columns=['betw_value'])
    between_size.reset_index(drop=True, inplace=True)
    
    ### Uncomment to exagerate nodes attribute changes
    ### i.e. result is significantly different nodes
    #e_cent_size = e_cent_size*150
    #age_rank_size = page_rank_size*1000
    #degree_size = degree_size*2
    #between_size = between_size*1000
    
    dfs = [e_cent_size,page_rank_size,degree_size,between_size]
    df = pd.concat(dfs, axis=1)
    ## Uncomment to exagerate nodes attribute changes
    # an_df = df.copy(deep=True)
    # Comment out section up to the #### if exagerating node attribute changes
    df = pd.concat(dfs, axis=1)
    cols = list(df.columns)
    an_arr = df.to_numpy(copy=True)
    #### Scale data so each metric is comparable.
    scaler = StandardScaler()
    an_scaled = scaler.fit_transform(an_arr)
    an_df = pd.DataFrame(an_scaled)
    an_st = an_df.copy(deep=True)
    an_st.columns = cols
    an_df.columns = cols
    an_mins = list(an_df.min())
    #### As normalisation around zero - making everything into positive numbers
    for i in range(len(an_mins)):
        an_df[cols[i]] -= an_mins[i] - 1
        an_df[cols[i]] *= 12
    ####
    an_df.columns = ['cent_st_val', 'rank_st_val', 'deg_st_val', 'betw_st_val']
    col_names = ['e_cent_col', 'rank_col', 'deg_col', 'betw_col']
    for i in range(len(dfs)):
        ### colour the nodes....
        col_out = node_color_picker(dfs[i], col_names[i])
        ### re-creating 'df' now with the colours...
        df = pd.concat([df, col_out], axis=1)
    #### bringing together original and scaled versions of dataframes
    full_an_df = pd.concat([df, an_df], axis=1)
    
    return full_an_df

In [8]:
#Creat the elements parameter to fit into the 

def create_cyto_graph(OrgCode, an_df,G):
    #G = create_nx_graph(OrgCode)
    nodeData = pd.read_csv('node_list.csv', low_memory=False)
    edgeData = pd.read_csv('edge_list/edge_list_' +OrgCode+'.csv', low_memory=False)
    nodeData = nodeData.astype(str)
    nodeData = nodeData[nodeData.ID.isin(list(G))]
    
    edgeData = edgeData.astype(str)
    nodes_list = list()
    for i in range(len(nodeData)):
        node = {
                "data": {"id": nodeData.iloc[i,0], 
                         "label": nodeData.iloc[i,1],
                         "cent_val": an_df.iloc[i,8],
                         "cent_col": an_df.iloc[i,4],
                         "rank_val": an_df.iloc[i,9],
                         "rank_col": an_df.iloc[i,5],
                         "deg_val": an_df.iloc[i,10],
                         "deg_col": an_df.iloc[i,6],
                         "betw_val": an_df.iloc[i,11],
                         "betw_col": an_df.iloc[i,7]}
                
            }
        nodes_list.append(node)
    
    edges_list = list()
    for j in range(len(edgeData)):
        edge = {
                "data": {"source": edgeData.iloc[j,0], 
                         "target": edgeData.iloc[j,1],
                         "weight": edgeData.iloc[j,2]}
            }
        edges_list.append(edge)
    
    elements = nodes_list + edges_list
    return elements


In [10]:
#set up the style sheet and starting graph for the webapp

orgcode = 'R0A'
G = create_nx_graph(orgcode)
full_an_df = create_analysis(G)
#### Extending the elements that need to go into the Cytoscape object
elements = create_cyto_graph(orgcode, full_an_df,G)

default_stylesheet=[
            {'selector': 'node',
                'style': {
                        'width': 'data(cent_val)', # these are now relative to the centrality value 
                        'height': 'data(cent_val)', # See Var Exp -> Elements -> Data (dict) -> cent_val
                        'background-color': 'data(cent_col)', # SEE HOW THESE ARE UPDATED IN CALLBACK FN change_metric
                        'content': 'data(label)',
                        'font-size': '40px',
                        'text-outline-color': 'white',
                        'text-outline-opacity': '1',
                        'text-outline-width': '8px',
                    }
                },
            {'selector': 'edge',
                'style': {
                        'line-color': 'white'
                    }
                }
        ]


In [11]:
#####################################
### Step 3 - App. Initialisation ####
#####################################
cyto.load_extra_layouts()
app = dash.Dash(__name__)
server = app.server

############################
### Step 4 - App. Layout ###
############################

app.layout = html.Div([
                cyto.Cytoscape(
                    id='cyto-graph',
                    className='net-obj',
                    elements=elements,
                    responsive = True,
                    style={'width':'100%', 'height':'800px'},
                    layout={'name': 'cose',
                            'padding': 5,
                            'nodeRepulsion': '10000',
                            'gravityRange': '1.0',
                            'nestingFactor': '0.4',
                            'edgeElasticity': '100',
                            'idealEdgeLength': '600',
                            'nodeDimensionsIncludeLabels': 'true',
                            'numIter': '300',
                            },
                    stylesheet=default_stylesheet
                    ),
                    html.Br(),
                    
                    html.Br(),
                dcc.Dropdown(
                    id='OrgCode-metric',
                    # List of dicts; keys: 'label' is human readable
                    # 'value' is a column name
                    options=[
                        {'label': 'ROA', 'value': 'R0A'},
                        {'label': 'R0B', 'value': 'R0B'},
                        {'label': 'R0D', 'value': 'R0D'},
                        {'label': 'R1F', 'value': 'R1F'},
                        {'label': 'R1H', 'value': 'R1H'},
                        {'label': 'R1K', 'value': 'R1K'},
                        {'label': 'RA2', 'value': 'RA2'},
                        {'label': 'RA3', 'value': 'RA3'},
                        {'label': 'RA4', 'value': 'RA4'},
                        {'label': 'RA7', 'value': 'RA7'},
                        {'label': 'RA9', 'value': 'RA9'},
                        {'label': 'RAE', 'value': 'RAE'},
                        {'label': 'RAJ', 'value': 'RAJ'},
                        {'label': 'RAL', 'value': 'RAL'},
                        {'label': 'RAP', 'value': 'RAP'},
                        {'label': 'RAS', 'value': 'RAS'},
                        {'label': 'RAX', 'value': 'RAX'},
                        {'label': 'RBD', 'value': 'RBD'},
                        {'label': 'RBK', 'value': 'RBK'},
                        {'label': 'RBL', 'value': 'RBL'},
                        {'label': 'RBN', 'value': 'RBN'},
                        {'label': 'RBS25', 'value': 'RBS25'},
                        {'label': 'RBT', 'value': 'RBT'},
                        {'label': 'RBZ', 'value': 'RBZ'},
                        {'label': 'RC9', 'value': 'RC9'},
                        {'label': 'RCB', 'value': 'RCB'},
                        {'label': 'RCD', 'value': 'RCD'},
                        {'label': 'RCF', 'value': 'RCF'},
                        {'label': 'RCU', 'value': 'RCU'},
                        {'label': 'RCX', 'value': 'RCX'},
                        {'label': 'RD1', 'value': 'RD1'},
                        {'label': 'RD8', 'value': 'RD8'},
                        {'label': 'RDE', 'value': 'RDE'},
                        {'label': 'RDU', 'value': 'RDU'},
                        {'label': 'REF', 'value': 'REF'},
                        {'label': 'REM', 'value': 'REM'},
                        {'label': 'RF4', 'value': 'RF4'},
                        {'label': 'RFF', 'value': 'RFF'},
                        {'label': 'RFR', 'value': 'RFR'},
                        {'label': 'RGN', 'value': 'RGN'},
                        {'label': 'RGR', 'value': 'RGR'},
                        {'label': 'RGT', 'value': 'RGT'},
                        {'label': 'RH5', 'value': 'RH5'},
                        {'label': 'RHM', 'value': 'RHM'},
                        {'label': 'RHQ', 'value': 'RHQ'},
                        {'label': 'RHU', 'value': 'RHU'},
                        {'label': 'RHW', 'value': 'RHW'},
                        {'label': 'RJ1', 'value': 'RJ1'},
                        {'label': 'RJ2', 'value': 'RJ2'},
                        {'label': 'RJ6', 'value': 'RJ6'},
                        {'label': 'RJ7', 'value': 'RJ7'},
                        {'label': 'RJC', 'value': 'RJC'},
                        {'label': 'RJE', 'value': 'RJE'},
                        {'label': 'RJL', 'value': 'RJL'},
                        {'label': 'RJN', 'value': 'RJN'},
                        {'label': 'RJR', 'value': 'RJR'},
                        {'label': 'RK5BC', 'value': 'RK5BC'},
                        {'label': 'RK9', 'value': 'RK9'},
                        {'label': 'RKB', 'value': 'RKB'},
                        {'label': 'RKE', 'value': 'RKE'},
                        {'label': 'RL4', 'value': 'RL4'},
                        {'label': 'RLQ', 'value': 'RLQ'},
                        {'label': 'RLT', 'value': 'RLT'},
                        {'label': 'RM1', 'value': 'RM1'},
                        {'label': 'RM3', 'value': 'RM3'},
                        {'label': 'RMC', 'value': 'RMC'},
                        {'label': 'RMP', 'value': 'RMP'},
                        {'label': 'RN3', 'value': 'RN3'},
                        {'label': 'RN5', 'value': 'RN5'},
                        {'label': 'RN7', 'value': 'RN7'},
                        {'label': 'RNA', 'value': 'RNA'},
                        {'label': 'RNN', 'value': 'RNN'},
                        {'label': 'RNQ', 'value': 'RNQ'},
                        {'label': 'RNS', 'value': 'RNS'},
                        {'label': 'RNZ', 'value': 'RNZ'},
                        {'label': 'RP5', 'value': 'RP5'},
                        {'label': 'RPA', 'value': 'RPA'},
                        {'label': 'RQ3', 'value': 'RQ3'},
                        {'label': 'RQM', 'value': 'RQM'},
                        {'label': 'RQW', 'value': 'RQW'},
                        {'label': 'RQX', 'value': 'RQX'},
                        {'label': 'RR7', 'value': 'RR7'},
                        {'label': 'RR8', 'value': 'RR8'},
                        {'label': 'RRK', 'value': 'RRK'},
                        {'label': 'RRV', 'value': 'RRV'},
                        {'label': 'RTD', 'value': 'RTD'},
                        {'label': 'RTF', 'value': 'RTF'},
                        {'label': 'RTG', 'value': 'RTG'},
                        {'label': 'RTH', 'value': 'RTH'},
                        {'label': 'RTP', 'value': 'RTP'},
                        {'label': 'RTR', 'value': 'RTR'},
                        {'label': 'RTX', 'value': 'RTX'},
                        {'label': 'RVJ', 'value': 'RVJ'},
                        {'label': 'RVR', 'value': 'RVR'},
                        {'label': 'RVV', 'value': 'RVV'},
                        {'label': 'RW6', 'value': 'RW6'},
                        {'label': 'RWA', 'value': 'RWA'},
                        {'label': 'RWD', 'value': 'RWD'},
                        {'label': 'RWE', 'value': 'RWE'},
                        {'label': 'RWF', 'value': 'RWF'},
                        {'label': 'RWG', 'value': 'RWG'},
                        {'label': 'RWH', 'value': 'RWH'},
                        {'label': 'RWJ', 'value': 'RWJ'},
                        {'label': 'RWP', 'value': 'RWP'},
                        {'label': 'RWW', 'value': 'RWW'},
                        {'label': 'RWY', 'value': 'RWY'},
                        {'label': 'RX1', 'value': 'RX1'},
                        {'label': 'RXC', 'value': 'RXC'},
                        {'label': 'RXF', 'value': 'RXF'},
                        {'label': 'RXL', 'value': 'RXL'},
                        {'label': 'RXN', 'value': 'RXN'},
                        {'label': 'RXP', 'value': 'RXP'},
                        {'label': 'RXQ', 'value': 'RXQ'},
                        {'label': 'RXR', 'value': 'RXR'},
                        {'label': 'RXW', 'value': 'RXW'},
                        {'label': 'RYJ', 'value': 'RYJ'},
                        {'label': 'RYR', 'value': 'RYR'},
                    ],
                        
                    clearable=False, # Always a value in drop down menu
                    multi=False,
                    value='R0A', # inital data to display
                    style={'width': '400px'}
                    ),
                    dcc.Slider(0,20,5,
                               value = 10,
                               id='my-slider-weight' )
                ])
               

###################################
### Step 5 - App Callback Funcs ###
###################################

##### Here is were we use the input for the drop down.
##### to determine the styling on the graph which color and
#####  metric value we should be using. 
@app.callback(Output(component_id= 'cyto-graph', component_property='elements'),
               # i.e., return from func below
            [Input(component_id= 'OrgCode-metric', component_property='value')]) # i.e., func below input
def change_metric(OrgValue):
    G = create_nx_graph(OrgValue)
    full_an_df = create_analysis(G)
#### Extending the elements that need to go into the Cytoscape object
    elements = create_cyto_graph(OrgValue, full_an_df,G)
    return elements

##################################
### Step 6 - App. Run Command ####
##################################

if __name__ == "__main__":
    app.run(jupyter_mode="external", debug=True, use_reloader=False)
    

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.